In [32]:
#Required libaries
from datetime import datetime, timezone, timedelta
import csv
import sys, os
import glob
import tweepy
from configparser import ConfigParser
from itertools import chain
#import tkinter

In [33]:
def config_reader() :
    try:
        config_object = ConfigParser()
        files = glob.glob("credentials.cfg")
        config_object.read(files)
        return config_object
    except KeyError as k:
        print(f"Key is missing in the config file {k}")
   

In [34]:
def tweepyAuthenticate():
    config_object = config_reader()
    auth = tweepy.OAuthHandler(config_object.get('TWITTER', 'consumer_key'), config_object.get('TWITTER', 'consumer_secret'))
    auth.set_access_token(config_object.get('TWITTER', 'access_token'), config_object.get('TWITTER', 'access_token_secret'))
    api = tweepy.API(auth)
    # test authentication
    try:
        api.verify_credentials()
        print("Authentication OK")
        return api, config_object
    except:
        print("Error during authentication")

In [35]:
def getLatestFile(folder_path):   
    #folder_path = r'C:\Users\Ron\Desktop\Test'
    file_type = r'/*csv'
    files = glob.glob(folder_path + file_type)
    max_file = max(files, key=os.path.getctime)

    #print(max_file)
    return max_file

In [36]:
def getSearchDateTime(filepath):
    print(os.getcwd())
    #os.chdir('data/twitterFavorite')
    #print(os.getcwd())
    if len(glob.glob(os.getcwd()+filepath+r"/*.csv")) > 0:
        filename = getLatestFile(os.getcwd()+filepath)
        print(filename)
        filetime= filename[filename.rfind('_')+1:filename.rfind('.csv')]
        print(type(datetime.strptime(filetime, '%Y%m%d%H%M')))
        print(datetime.strptime(filetime, '%Y%m%d%H%M').replace(tzinfo=timezone.utc))
        return datetime.strptime(filetime, '%Y%m%d%H%M').replace(tzinfo=timezone.utc)
    else:
        return datetime.now(timezone.utc) - timedelta(hours=1)

In [37]:
def write_csv(list_write, filename):
    keys = list_write[0].keys()
    fileGenerated = datetime.now(timezone.utc).strftime("%Y%m%d%H%M")
    #with open(r'data\twitterFavorite\twitter_favorite_'+fileGenerated+'.csv', 'w', newline='') as output_file:
    with open(filename+fileGenerated+'.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, keys)
        dict_writer.writeheader()
        dict_writer.writerows(list_write)

### Function favoriteTweet to mark tweets as favorite based on search criteria

In [38]:
def favoriteTweet():
    #search_list = ["#python", "#matplotlib", "#pandas", "#streamlit","#seaborn"]
    #search = "#python"
    lang = "en"
    numberOfTweets = 20
    list_write = []
    datetime_object = getSearchDateTime(r"/data/twitterFavorite")
    api, config_object = tweepyAuthenticate()
    search_list = config_object.get('TWITTER', 'search_list_fav').split(',')
    #print(search_list)
    #print(type(search_list))
    print("date beyond which search to start", datetime_object)
    for search in search_list:
        #print(f"searching tag {search}")
        for tweet in tweepy.Cursor(api.search_tweets, search, lang = lang, result_type = 'recent').items(numberOfTweets):
            try:

                if tweet.created_at > datetime_object :
                    #print(tweet.user.screen_name)
                    #print(tweet.favorited)
                    #print("Tweet created time ",tweet.created_at)
                    try:
                        url = tweet.entities['urls'][0]['url']
                        expanded_url = tweet.entities['urls'][0]['expanded_url']
                    except IndexError:
                        url = ""
                        expanded_url = ""
                    
                    tweet.favorite()
                    list_write.append({"id":tweet.id,"created_at":tweet.created_at.strftime("%Y-%m-%d %H:%M:%S"),\
                                       "tag":search,"screen_name":tweet.user.screen_name,\
                                       "text":(tweet.text).replace('\n','').encode('utf-8'), "url":url,\
                                       "expanded_url":expanded_url,\
                                       "favorited_at":datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")})
                    #print('favorited the tweet')
            except tweepy.TweepyException as e:
                print(e.api_errors)
            except StopIteration:
                break

    #print(list_write)
    write_csv(list_write, r'data/twitterFavorite/twitter_favorite_')

In [39]:
def saveTweet():
    datetime_object = getSearchDateTime(r"/data/twitterRetweet")
    api, config_object = tweepyAuthenticate()
    search_list = config_object.get('TWITTER', 'search_list_save').split(',')
    final_tweets = []
    #search_list = ["#python", "#matplotlib", "#pandas", "streamlit", "#seaborn"]
    screen_name = config_object.get('TWITTER', 'screenName')
    print(f"Follow (accounts followed by {screen_name})")
    for page in tweepy.Cursor(api.get_friends, screen_name=screen_name,
                              count=200).pages(10):
        for user in page:
        
            name = f"{user.id} - {user.name} (@{user.screen_name})"
            #followers.append(name)

            print(f"{user.screen_name}")
            try:
                #api.create_friendship(user_id = user.id, follow = False)
                timeline = api.user_timeline(user_id = user.id, screen_name = user.screen_name,\
                                             tweet_mode="extended", exclude_replies = True,include_rts = False)
                #print(timeline)
                for index, search in enumerate(search_list):
                    print(f"search item {search}")
                    textonly_tweets = [{"id":tweet.id,"created_at":tweet.created_at.strftime("%Y-%m-%d %H:%M:%S"),\
                                               "tag":search,"screen_name":tweet.user.screen_name,\
                                               "text":(tweet.full_text).replace('\n','').encode('utf-8'), "url":'',\
                                               "expanded_url":'',\
                                               "tweeted_at":datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M:%S")} 
                                       for tweet in timeline 
                                       if (search in tweet.full_text and tweet.created_at > datetime_object)]

                    #print("textonly_tweets =",textonly_tweets)
                    if len(textonly_tweets) > 0:
                        for retweet in textonly_tweets:
                            if retweet['tag'][0] != '#':
                                tag = '#'+retweet['tag'].strip()
                                print(tag)
                            else:
                                tag = retweet['tag'].strip()
                            try:
                                api.update_status(status = f"{tag} #tweet @savetonotion", in_reply_to_status_id = retweet['id'], auto_populate_reply_metadata=True)
                                #pass
                            except tweepy.TweepyException as e:
                                print(e.api_errors)
                            except StopIteration:
                                break
                            except:
                                print("Unknown error occured")
                                continue
                        final_tweets.append(textonly_tweets)
            except tweepy.TweepyException as e:
                print(e.api_errors)
                continue
        print(len(page))


    print("length is ",len(final_tweets))
    #print(final_tweets)
    #print(list(chain.from_iterable(final_tweets)))
    if len(final_tweets) > 0:
        write_csv(list(chain.from_iterable(final_tweets)),r'data/twitterRetweet/twitter_retweet_')
    else:
        print("no record present")

### Below code to follow all IDs following my ID

In [40]:
def followFollowers():
    followers = []
    
    api, config_object = tweepyAuthenticate()
    screen_name = config_object.get('TWITTER', 'screenName')#"mohdatif1987"
    print(f"Followers (accounts who follow {screen_name})")
    for page in tweepy.Cursor(api.get_followers, screen_name=screen_name,
                              count=100).pages(10):
        for user in page:
            name = f"{user.id} - {user.name} (@{user.screen_name})"
            followers.append(name)
            if not user.following:
                print(f"{user.screen_name}")
                try:
                    api.create_friendship(user_id = user.id, follow = False)
                except tweepy.TweepyException as e:
                    print(e.api_errors)
                    continue
                print(f"Started following {user.screen_name}")
        print(len(page))

    print(f"Followers: {len(followers)}")
    #print(followers)

In [41]:
if __name__ == "__main__":
    followFollowers()
    favoriteTweet()
    saveTweet()

E:\python\tweeter_api\twitter\twitter_bot_project
E:\python\tweeter_api\twitter\twitter_bot_project/data/twitterRetweet\twitter_retweet_202212280530.csv
<class 'datetime.datetime'>
2022-12-28 05:30:00+00:00
Authentication OK
Follow (accounts followed by mohdatif1987)
OlehPylyp
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
Rizwan41751028
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
ATsobaza
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
Ali_561
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
ramiboutas
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
dong_marcelo
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
clcoding
search item #py

search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
murraycorwin1
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
Dunsinagb
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
DataTechSpace
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
CHOTIWALA
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
KMbappe
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
_townee
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
deephaven
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
nramind
search item #python
search item #matplotlib
search item #panda

search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
mariatta
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
llanga
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
treyhunner
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
MongoDB
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
Aloksharmaaicc
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
mariadb
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
anacondainc
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
IPythonDev
search item #python
search item #matplotlib
search item #pandas

search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
AdvocateAnupamS
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
thewire_in
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
barcouncilindia
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
thecaravanindia
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
MHiesboeck
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
bbelderbos
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
bascodes
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
akshay_pachaar
search item #python
search item #matplo

search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
sspendol
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
vaibhaw_g
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
KnownAsMarkus
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
JLOracle
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
InsumSolutions
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
jasminfluri
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
thatjeffsmith
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
LegalTL
search item #python
search item #matplotlib
search it

search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
Kisanektamorcha
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
OfficialSauravD
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
_YogendraYadav
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
laluprasadrjd
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
NooriKhanINC
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
raihanrvadra
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
udhamdelhigov
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
ChetanNature
search item #python
search ite

search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
jollyjohri
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
Gop_krishan
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
khanshain07
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
AshrafFem
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
thelede_in
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
srivatsayb
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
PunjabPoliceInd
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
AditiSinghRBL
search item #python
search item #matplotlib
sea

search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
HimuNsui
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
mayank_sxn
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
manaschopra3
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
ZaidAli81499746
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
BibiPakhii
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
joymatts
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
VishalLochab6
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
SridharJayshree
search item #python
search item #matplotlib
s

search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
cnni
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
drajoykumar
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
ShatruganSinha
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
DeependerSHooda
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
INCTharoorian
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
azharflicks
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
PIB_India
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
YUVSTRONG12
search item #python
search item #matplotlib
sear

search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
BeingSalmanKhan
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
iamsrk
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
SrBachchan
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
drcpjoshi
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
AnupamPKher
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
ashokgehlot51
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
SachinPilot
search item #python
search item #matplotlib
search item #pandas
search item streamlit
search item #seaborn
thekiranbedi
search item #python
search item #matplotlib
searc